In [6]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate("lanaagent-firebase-adminsdk-fbsvc-3ed45f99b9.json")
firebase_admin.initialize_app(cred)

db=firestore.client()
#db.collection('persons').add({'name':'John', 'age':30})

In [7]:
from datetime import datetime, timedelta

# Assume db = firestore.client() has been executed and is available

def log_persistence(log_entry_data: dict, is_authenticated: bool) -> bool:
    """
    Saves a structured log entry to the Google Cloud Persistence Store (Firestore).
    Includes a Contact Gate check and an auto-generated timestamp.
    """
    
    # 1. Structural Check: The Contact Gate (for ethical protection)
    if log_entry_data.get('key') == 'contact_info' and not is_authenticated:
        print("ACCESS DENIED: Cannot share contact info for anonymous user.")
        # We do not log the entry to prevent accidental data leakage
        return False
    
    try:
        # 2. Add Required Metadata (Timestamp)
        # Create a new dictionary to ensure we always log a timestamp
        data_to_log = log_entry_data.copy()
        data_to_log['timestamp'] = datetime.utcnow()
        
        # 3. Cloud Write: Writing the data to the 'logs' collection in Firestore
        db.collection('logs').add(data_to_log)
        
        print(f"SUCCESS: Logged data to Firestore: {log_entry_data.get('key')}")
        return True
        
    except Exception as e:
        print(f"FAILURE: Could not write to Firestore. Error: {e}")
        return False

In [8]:
from datetime import datetime, timedelta

# Assume db = firestore.client() is available

def query_persistence(task_id: str) -> dict:
    """
    Queries the Persistence Store (Firestore) for a specific task's history 
    and checks if the Contradiction Flag should be raised.
    
    Args:
        task_id: The unique identifier (Task Key) of the task to search for.

    Returns:
        A dictionary containing the check result and failure count.
    """
    
    # 1. Define the search window (e.g., last 3 days)
    # The FOE only cares about recent, consecutive failures.
    three_days_ago = datetime.now() - timedelta(days=3)
    
    # 2. Query Firestore: Get logs for this task_id where status was 'O' (Avoided) 
    # and the entry is recent.
    logs_ref = db.collection('logs').where('key', '==', task_id)
    
    # We will refine this query and logic in the next step, but for now, 
    # we simulate the core check:
    
    # --- YOUR TASK IS HERE ---
    # Implement the final structural check (The Contradiction Flag logic)
    
    # Simulate a successful query that found 4 failures in a row
    consecutive_failure_count = 4 

    # --- FINAL STRUCTURAL STEP: THE CONTRADICTION FLAG LOGIC ---
    # The flag is raised if the failure count is 3 or more.
    flag_raised = consecutive_failure_count >= 3
    
    # Replace the 'pass' below with the conditional check.
    pass

    return {
        "task_id": task_id,
        "consecutive_failures": consecutive_failure_count,
        "flag_raised": flag_raised
    }

In [9]:
# The log_entry_data dictionary ready to be passed to log_persistence()
log_entry_data_test_case = {
    # CATEGORY I: The metadata for the log entry
    "category": "Avoidance/Failure",
    "key": "Contradiction_Flag_Event_Social_Crusade",
    
    # CATEGORY II: The actual structured data (value)
    "value": {
        # This is the 'what' happened
        "Failure_Type": "Redirected from goals to low-value, low-effort activity",
        
        # This is the 'why' (Justification Log)
        "Justification_Log": "Efforts to engage in unproductive social interraction (High-Energy Avoidance)",
        
        # This is the structural fix (Corrective Action)
        "Corrective_Action": "Implement Social Interaction Protocol (SIP): 30-minute daily social block enforced.",
        
        # Contextual data
        "Incomplete_Tasks": ["Implement log_persistence tool", "Refine bujo_reader function"]
    },
    
    # CATEGORY III: The source of the log
    "source": "User-Logged via UI"
}

In [10]:
log_persistence(log_entry_data_test_case, is_authenticated=True)

C:\Users\Owner\AppData\Local\Temp\ipykernel_19224\2565193077.py:21: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  data_to_log['timestamp'] = datetime.utcnow()


SUCCESS: Logged data to Firestore: Contradiction_Flag_Event_Social_Crusade


True